In [47]:
import json
import os
import re
from pathlib import Path
import glob
from collections import OrderedDict
from operator import itemgetter
import math
import shutil
# for all the files in folder and subfolders, if files ends with "_results" print the file name
folder = r"C:\OneDrive_CIRAD\OneDrive - Cirad\ML"
name_list = []
full_path_list = []
not_found_csv = {}
for path, subdirs, files in os.walk(folder):
    for name in files:
        if name.endswith("_results.json"):
            # if there is a subdirectory with the same name as the file without _results.json
            # then print the path of the file
            name_wo_time = name[:-13]
            if "RMSE" in name_wo_time:
                canonical_name = "_".join(name_wo_time.split("_")[:-1])
            else:
                canonical_name = name_wo_time

            if os.path.isdir(os.path.join(path, name_wo_time)) and "Paprica" not in canonical_name and "Raisin" not in canonical_name:
                name_list.append(canonical_name)
                full_path_list.append((os.path.join(path, name), canonical_name))
            else:
                pass
                # print("No folder for", canonical_name)

for path, subdirs, files in os.walk(r"C:\OneDrive_CIRAD\OneDrive - Cirad\data"):
    for name in files:
        if name.endswith("_results.json"):
            # if there is a subdirectory with the same name as the file without _results.json
            # then print the path of the file
            name_wo_time = name[:-13]
            if "RMSE" in name_wo_time:
                canonical_name = "_".join(name_wo_time.split("_")[:-1])
            else:
                canonical_name = name_wo_time

            if os.path.isdir(os.path.join(path, name_wo_time)) and "Paprica" not in canonical_name and "Raisin" not in canonical_name:
                name_list.append(canonical_name)
                full_path_list.append((os.path.join(path, name), canonical_name))
            else:
                pass
                # print("No folder for", canonical_name)


def clean_result(result, path, name, csv_path):
    # print(result)
    if "targetRMSE" in result:
        del result["targetRMSE"]
    keys = ["seed", "RMSE", "MAE", "MAPE", "R2", "MSE", "MedAE", "EVS"]
    for k in keys:
        if k in result:
            try:
                if k == "seed":
                    result[k] = int(result[k])
                else:
                    result[k] = float(result[k])
            except TypeError as e:
                print(e)
                print(k, result[k])
    close = False
    try:
        close = math.isclose(result["RMSE"], (result["MSE"]*result["MSE"]))
    except:
        print("Impossible to compute close")
        # print(result)
        print(result, path, name)

    if close:
        tmp = result["RMSE"]
        result["RMSE"] = result["MSE"]
        result["MSE"] = tmp
        # print("inverted RMSE and MSE for", result["name"])

    # check csv file
    csv_file_path = os.path.join(csv_path, result["model"] + ".csv")
    found_csv = False

    if os.path.isfile(csv_file_path):
        # print("CSV file found for", name)
        new_path = os.path.join("csv", name, result["model"] + ".csv")
        # cp file to new folder
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        shutil.copy(csv_file_path, new_path)
        found_csv = True
    else:
        not_found_csv[csv_file_path] = result["model"]
        # not_found_csv.append(csv_file_path)
        print("CSV file not found for", name, result["model"])

    return result


results = {}
for path, name in full_path_list:
    if name not in results:
        results[name] = []
    json_results = json.load(open(path))
    csv_path = path.replace("_results.json", "")
    if isinstance(json_results, list):
        for result in json_results:
            if isinstance(result, dict):
                results[name].append(clean_result(result, path, name, csv_path))
            elif isinstance(result, list):
                for r in result:
                    if isinstance(r, str):
                        continue
                    results[name].append(clean_result(r, path, name, csv_path))
            else:
                print("unknown type", type(result))
    elif isinstance(json_results, dict):
        for k, v in json_results.items():
            if isinstance(v, dict):
                results[name].append(clean_result(v, path, name, csv_path))
            elif isinstance(v, list):
                for r in v:
                    if isinstance(r, str):
                        continue
                    results[name].append(clean_result(r, path, name, csv_path))
            else:
                print("unknown type", type(v))
    else:
        print("unknown type", type(json_results))


print(len(not_found_csv), "CSV files not found")
print(json.dumps(not_found_csv, indent=4))

for k, v in results.items():
    # print(k, v)
    # break
    try:
        results[k] = sorted(v, key=itemgetter("RMSE"))
    except Exception as e:
        print("Problem with", k)
        print(e)

json.dump(results, open("all_results.json", "w"), indent=4)


for k, v in results.items():
    print(k, len(v))

print("SUM", sum([len(v) for v in results.values()]))

CSV file not found for ALPINE_C_424_Murguzur sklearn.cross_decomposition._pls.PLSRegression--NoSpl-CV_4_1-NoAug-PP_12_61574-42-23-03-01_10-19-39
CSV file not found for ALPINE_C_424_Murguzur sklearn.cross_decomposition._pls.PLSRegression--NoSpl-CV_4_1-NoAug-PP_3_59251-42-23-03-01_10-19-39
CSV file not found for Cassava_TBC_3393_Sanchez sklearn.cross_decomposition._pls.PLSRegression--NoSpl-CV_4_1-NoAug-PP_12_61574-42-23-03-01_10-20-03
CSV file not found for Cassava_TBC_3393_Sanchez sklearn.cross_decomposition._pls.PLSRegression--NoSpl-CV_4_1-NoAug-PP_3_59251-42-23-03-01_10-20-03
CSV file not found for LUCAS_SOCgrassland_4096_Nocita sklearn.cross_decomposition._pls.PLSRegression--NoSpl-CV_4_1-NoAug-PP_3_59251-42-23-03-01_10-21-30
CSV file not found for LUCAS_SOCgrassland_4096_Nocita sklearn.cross_decomposition._pls.PLSRegression--NoSpl-CV_4_1-NoAug-PP_12_61574-42-23-03-01_10-21-30
CSV file not found for Meat_Fat_215_Borggaard sklearn.cross_decomposition._pls.PLSRegression--NoSpl-CV_4_1-No